# 🚇 Akbank Python ve Yapay Zekaya Giriş Bootcamp Projesi: Sürücüsüz Metro Simülasyonu (Rota Optimizasyonu)

In [1]:
from collections import defaultdict, deque
import heapq
from typing import Dict, List, Set, Tuple, Optional

## 1-İstasyon Sınıfı

### 1.1-`__init__` Fonksiyonu

In [2]:
class Istasyon:
    def __init__(self, idx: str, ad: str, hat: str):
        self.idx = idx
        self.ad = ad
        self.hat = hat
        self.komsular: List[Tuple['Istasyon', int]] = []  # (istasyon, süre) tuple'ları

Her istasyon nesnesinin 4 adet değişkeni var:
- 1-İstasyonun indeksi (idx)
    - M1-K1-T1 gibi
- 2-İstasyon adı (ad) bağlı olduğu hat 
    - Mavi-Kırmızı-Turuncu 
- 3-İstasyonun List halinde ona bağlı olan komşu istasyonları ve süreleri
    - Komşu istasyonlar Tuple içinde İstasyon objesinden üretiliyor süre değeri de saklanıyor.

In [3]:
istasyon = Istasyon("M1", "AŞTİ", "Mavi Hat")
print(istasyon)

Kodu test ederken obje olarak döndüğünden `__repr__` eklemesine ihtiyaç duyuldu

### 1.2-`__repr__` Fonksiyonu

In [4]:
class Istasyon(Istasyon):
    def __repr__(self):
        return f"Istasyon({self.idx}, {self.ad}, {self.hat})"  

`__repr__` metodu, objenin anlaşılır bir şekilde string olarak temsil edilmesini sağlar.

In [5]:
istasyon1 = Istasyon("M1", "AŞTİ", "Mavi Hat")
print(istasyon1)

Istasyon(M1, AŞTİ, Mavi Hat)


Örnek çıktı yukarıdaki gibidir.

### 1.3-komsu_ekle fonksiyonu

In [6]:
class Istasyon(Istasyon):
    def komsu_ekle(self, istasyon: 'Istasyon', sure: int):
            self.komsular.append((istasyon, sure))

İstasyonlara komşu istasyon eklemek için.

Eklenecek istasyon objesini ve süreyi alır. self.komsular'a ekler

In [7]:
istasyon1 = Istasyon("M1", "AŞTİ", "Mavi Hat")
istasyon2=Istasyon("M2","Kızılay","Mavi Hat")
istasyon1.komsu_ekle(istasyon2,5)
print(istasyon1.komsular)

[(Istasyon(M2, Kızılay, Mavi Hat), 5)]


Örnek kullanımı ve sonucu yukarıdaki gibidir.

## 2-Metro Ağı Sınıfı

### 2.1-`__init__` Fonksiyonu

In [8]:
class MetroAgi:
    def __init__(self):
        self.istasyonlar: Dict[str, Istasyon] = {}
        self.hatlar: Dict[str, List[Istasyon]] = defaultdict(list)

Metro Ağı Sınıfı tanımlanır. Metro Ağının:
- 1-Dictionary formatında istasyonları bulunur.
- 2-Dictionary formatında İstasyonların birbirine bağlandığı hatlar bulunur

### 2.2-`istasyon_ekle` fonksiyonu

In [9]:
class MetroAgi(MetroAgi):
    def istasyon_ekle(self, idx: str, ad: str, hat: str) -> None:
        if id not in self.istasyonlar:
            istasyon = Istasyon(idx, ad, hat)
            self.istasyonlar[idx] = istasyon
            self.hatlar[hat].append(istasyon)

Verilen istasyonu verilen hatta eklemek için kullanılır.

### 2.3-`baglanti_ekle` fonksiyonu

In [10]:
class MetroAgi(MetroAgi):
    def baglanti_ekle(self, istasyon1_id: str, istasyon2_id: str, sure: int) -> None:
        istasyon1 = self.istasyonlar[istasyon1_id]
        istasyon2 = self.istasyonlar[istasyon2_id]
        istasyon1.komsu_ekle(istasyon2, sure)
        istasyon2.komsu_ekle(istasyon1, sure)

İki istasyon arasındaki bağlantıyı tanımlamak için kullanılır.

### 2.4-`en_az_aktarma_bul` fonksiyonu (*BFS Algoritması*)

In [11]:
class MetroAgi(MetroAgi):
    def en_az_aktarma_bul(self, baslangic_id: str, hedef_id: str) -> Optional[List[Istasyon]]:
        try:
            # Eğer başlangıç istasyonu ağda yoksa None döndür
            if baslangic_id not in self.istasyonlar:
                print("Başlangıç istasyonu ağda bulunamadı.")
                return None
            
            # Eğer hedef istasyonu ağda yoksa None döndür
            if hedef_id not in self.istasyonlar:
                print("Hedef istasyonu ağda bulunamadı.")
                return None
            
            # Başlangıç ve Hedef istasyonu 
            baslangic = self.istasyonlar[baslangic_id]  
            hedef = self.istasyonlar[hedef_id]

            # Kuyruğa başlangıç istasyonu ve rota listesi eklenir
            kuyruk = deque([(baslangic, [baslangic])])  

            # Ziyaret edilen istasyonları tutmak için dictionary
            ziyaret_edildi = {baslangic}  

            while kuyruk:

                # Kuyruğun başındaki öğeyi al
                mevcut, rota = kuyruk.popleft()

                # Eğer hedefe ulaşmışsak rota döndürülür
                if mevcut == hedef:
                    return rota 
                
                # Mevcut istasyonun komşuları üzerinden gezin
                for komsu, _ in mevcut.komsular:  

                    # Eğer komşu istasyon daha önce ziyaret edilmemişse
                    if komsu not in ziyaret_edildi:

                        # Komşu istasyonu ziyaret edilmemişler listesine ekle
                        ziyaret_edildi.add(komsu)

                        # Komşu istasyonu rotaya ekle ve kuyruğa ekle
                        kuyruk.append((komsu, rota + [komsu]))  
            
            # Eğer hedefe ulaşılamazsa None döndürülür
            print("Hedefe ulaşılamadı!")
            return None
        except Exception as e:
                print(f"Beklenmedik bir hata oluştu: {e}")
                return None  # Herhangi başka bir hata oluşursa, hata mesajı verilir ve None döndürülür

BFS Algoritması:

- Başlangıç istasyonu ile bir kuyruk başlatılır ve bu istasyonun yolculuk rotası başlatılır.
- Ziyaret edilen istasyonlar bir küme içinde tutulur.
- Kuyruktan bir istasyon çıkarılır, bu istasyonun komşuları kontrol edilir.
- Eğer hedef istasyon bulunursa, o anki rota döndürülür.
- Eğer komşu istasyon daha önce ziyaret edilmemişse, ziyaret edilmişler listesine eklenir ve komşu istasyon rotaya eklenerek kuyruğa eklenir.

### 2.5-`en_hizli_rota_bul` fonksiyonu (A* Algoritması)

In [12]:
class MetroAgi(MetroAgi):
    def en_hizli_rota_bul(self, baslangic_id: str, hedef_id: str) -> Optional[Tuple[List[Istasyon], int]]:
        try:
            # Eğer başlangıç istasyonu ağda yoksa döndür
            if baslangic_id not in self.istasyonlar:
                print("Başlangıç istasyonu ağda bulunamadı.")
                return None
            
            # Eğer hedef istasyonu ağda yoksa döndür
            if hedef_id not in self.istasyonlar:
                print("Hedef istasyonu ağda bulunamadı.")
                return None
            
            # Eğer başlangıç istasyonu ağda yoksa None döndür
            baslangic = self.istasyonlar[baslangic_id]
            hedef = self.istasyonlar[hedef_id]

            # Ziyaret edilen istasyonlar
            ziyaret_edildi = set()  

            # Öncelik sırasına göre işlem yapmak için min-heap (priority queue) kullanıyoruz. Başlangıç istasyonu kuyruğa alınır.
            pq = [(0, id(baslangic), baslangic, [baslangic])] 

            while pq:

                # Öncelikli kuyruktan en küçük zamanlı öğeyi al
                zaman, _, mevcut, rota = heapq.heappop(pq)

                # Hedef istasyonu bulundu, rota ve süre döndürülür 
                if mevcut == hedef:
                    return rota, zaman  
                
                 # Eğer bu istasyon daha önce ziyaret edildiyse geç
                if mevcut in ziyaret_edildi:
                    continue 

                # İstasyonu ziyaret edilmiş olarak işaretle
                ziyaret_edildi.add(mevcut)  

                # Mevcut istasyonun komşularını geziyoruz
                for komsu, seyahatSüresi in mevcut.komsular:
                    
                    # Eğer komşu istasyon daha önce ziyaret edilmediyse
                    if komsu not in ziyaret_edildi:  
                        # Seyahat süresiyle birlikte komşu istasyonu kuyruğa ekle
                        heapq.heappush(pq, (zaman + seyahatSüresi, id(komsu), komsu, rota + [komsu]))
            # Eğer hedefe ulaşamazsak None döndürülür
            print("Hedefe ulaşılamadı!")
            return None  
        except Exception as e:
            print(f"Beklenmedik bir hata oluştu: {e}")
            return None  # Herhangi başka bir hata oluşursa, hata mesajı verilir ve None döndürülür

- Kuyruk boşalana kadar devam edilir. Kuyruktan her seferinde en küçük zamanlı (yani en kısa süreli) istasyon çıkarılır.
- Eğer bu istasyon hedefe eşitse, mevcut rota ve süre döndürülür.
- Eğer istasyon daha önce ziyaret edildiyse, bu istasyonu atlar ve bir sonraki istasyona geçilir.
- Her komşu istasyon için, komşuya ulaşma süresi eklenerek kuyruk güncellenir. Bu komşu istasyon, daha önce ziyaret edilmemişse kuyruğa eklenir.

## 3-Test Aşaması

In [13]:
metro = MetroAgi()

In [14]:
print(metro)

### 3.1-İstasyonlar ekleme

In [15]:
# Kırmızı Hat
metro.istasyon_ekle("K1", "Kızılay", "Kırmızı Hat")
metro.istasyon_ekle("K2", "Ulus", "Kırmızı Hat")
metro.istasyon_ekle("K3", "Demetevler", "Kırmızı Hat")
metro.istasyon_ekle("K4", "OSB", "Kırmızı Hat")

In [16]:
print(metro.istasyonlar)

{'K1': Istasyon(K1, Kızılay, Kırmızı Hat), 'K2': Istasyon(K2, Ulus, Kırmızı Hat), 'K3': Istasyon(K3, Demetevler, Kırmızı Hat), 'K4': Istasyon(K4, OSB, Kırmızı Hat)}


In [17]:
# Mavi Hat
metro.istasyon_ekle("M1", "AŞTİ", "Mavi Hat")
metro.istasyon_ekle("M2", "Kızılay", "Mavi Hat")  # Aktarma noktası
metro.istasyon_ekle("M3", "Sıhhiye", "Mavi Hat")
metro.istasyon_ekle("M4", "Gar", "Mavi Hat")

In [18]:
print(metro.istasyonlar)

{'K1': Istasyon(K1, Kızılay, Kırmızı Hat), 'K2': Istasyon(K2, Ulus, Kırmızı Hat), 'K3': Istasyon(K3, Demetevler, Kırmızı Hat), 'K4': Istasyon(K4, OSB, Kırmızı Hat), 'M1': Istasyon(M1, AŞTİ, Mavi Hat), 'M2': Istasyon(M2, Kızılay, Mavi Hat), 'M3': Istasyon(M3, Sıhhiye, Mavi Hat), 'M4': Istasyon(M4, Gar, Mavi Hat)}


In [19]:
# Turuncu Hat
metro.istasyon_ekle("T1", "Batıkent", "Turuncu Hat")
metro.istasyon_ekle("T2", "Demetevler", "Turuncu Hat")  # Aktarma noktası
metro.istasyon_ekle("T3", "Gar", "Turuncu Hat")  # Aktarma noktası
metro.istasyon_ekle("T4", "Keçiören", "Turuncu Hat")

In [20]:
print(metro.istasyonlar)

{'K1': Istasyon(K1, Kızılay, Kırmızı Hat), 'K2': Istasyon(K2, Ulus, Kırmızı Hat), 'K3': Istasyon(K3, Demetevler, Kırmızı Hat), 'K4': Istasyon(K4, OSB, Kırmızı Hat), 'M1': Istasyon(M1, AŞTİ, Mavi Hat), 'M2': Istasyon(M2, Kızılay, Mavi Hat), 'M3': Istasyon(M3, Sıhhiye, Mavi Hat), 'M4': Istasyon(M4, Gar, Mavi Hat), 'T1': Istasyon(T1, Batıkent, Turuncu Hat), 'T2': Istasyon(T2, Demetevler, Turuncu Hat), 'T3': Istasyon(T3, Gar, Turuncu Hat), 'T4': Istasyon(T4, Keçiören, Turuncu Hat)}


### 3.2-Bağlantıları ekleme

In [21]:
print(metro.istasyonlar["K1"].komsular)
print(metro.istasyonlar["K2"].komsular)
print(metro.istasyonlar["K3"].komsular)

[]
[]
[]


In [22]:
# Kırmızı Hat bağlantıları
metro.baglanti_ekle("K1", "K2", 4)  # Kızılay -> Ulus
metro.baglanti_ekle("K2", "K3", 6)  # Ulus -> Demetevler
metro.baglanti_ekle("K3", "K4", 8)  # Demetevler -> OSB

In [23]:
print(metro.istasyonlar["K1"].komsular)
print(metro.istasyonlar["K2"].komsular)
print(metro.istasyonlar["K3"].komsular)

[(Istasyon(K2, Ulus, Kırmızı Hat), 4)]
[(Istasyon(K1, Kızılay, Kırmızı Hat), 4), (Istasyon(K3, Demetevler, Kırmızı Hat), 6)]
[(Istasyon(K2, Ulus, Kırmızı Hat), 6), (Istasyon(K4, OSB, Kırmızı Hat), 8)]


In [24]:
# Mavi Hat bağlantıları
metro.baglanti_ekle("M1", "M2", 5)  # AŞTİ -> Kızılay
metro.baglanti_ekle("M2", "M3", 3)  # Kızılay -> Sıhhiye
metro.baglanti_ekle("M3", "M4", 4)  # Sıhhiye -> Gar

In [25]:
print(metro.istasyonlar["M1"].komsular)
print(metro.istasyonlar["M2"].komsular)
print(metro.istasyonlar["M3"].komsular)

[(Istasyon(M2, Kızılay, Mavi Hat), 5)]
[(Istasyon(M1, AŞTİ, Mavi Hat), 5), (Istasyon(M3, Sıhhiye, Mavi Hat), 3)]
[(Istasyon(M2, Kızılay, Mavi Hat), 3), (Istasyon(M4, Gar, Mavi Hat), 4)]


In [26]:
# Turuncu Hat bağlantıları
metro.baglanti_ekle("T1", "T2", 7)  # Batıkent -> Demetevler
metro.baglanti_ekle("T2", "T3", 9)  # Demetevler -> Gar
metro.baglanti_ekle("T3", "T4", 5)  # Gar -> Keçiören

In [27]:
print(metro.istasyonlar["T1"].komsular)
print(metro.istasyonlar["T2"].komsular)
print(metro.istasyonlar["T3"].komsular)

[(Istasyon(T2, Demetevler, Turuncu Hat), 7)]
[(Istasyon(T1, Batıkent, Turuncu Hat), 7), (Istasyon(T3, Gar, Turuncu Hat), 9)]
[(Istasyon(T2, Demetevler, Turuncu Hat), 9), (Istasyon(T4, Keçiören, Turuncu Hat), 5)]


###  3.3-Hat aktarma bağlantıları (aynı istasyon farklı hatlar)

In [28]:
print(metro.istasyonlar["K1"].komsular)
print(metro.istasyonlar["K3"].komsular)
print(metro.istasyonlar["M4"].komsular)

[(Istasyon(K2, Ulus, Kırmızı Hat), 4)]
[(Istasyon(K2, Ulus, Kırmızı Hat), 6), (Istasyon(K4, OSB, Kırmızı Hat), 8)]
[(Istasyon(M3, Sıhhiye, Mavi Hat), 4)]


In [29]:

metro.baglanti_ekle("K1", "M2", 2)  # Kızılay aktarma
metro.baglanti_ekle("K3", "T2", 3)  # Demetevler aktarma
metro.baglanti_ekle("M4", "T3", 2)  # Gar aktarma

In [30]:
print(metro.istasyonlar["K1"].komsular)
print(metro.istasyonlar["K3"].komsular)
print(metro.istasyonlar["M4"].komsular)

[(Istasyon(K2, Ulus, Kırmızı Hat), 4), (Istasyon(M2, Kızılay, Mavi Hat), 2)]
[(Istasyon(K2, Ulus, Kırmızı Hat), 6), (Istasyon(K4, OSB, Kırmızı Hat), 8), (Istasyon(T2, Demetevler, Turuncu Hat), 3)]
[(Istasyon(M3, Sıhhiye, Mavi Hat), 4), (Istasyon(T3, Gar, Turuncu Hat), 2)]


### 3.4-Test Senaryoları

<div>
<img src="../diagrams/istasyonlar.svg" width="750"/>
</div>

#### *Senaryo 1: AŞTİ'den OSB'ye*

In [ ]:
print("\n1. AŞTİ'den OSB'ye:")
rota = metro.en_az_aktarma_bul("M1", "K4")
if rota:
    print("En az aktarmalı rota:", " -> ".join(i.ad for i in rota))

sonuc = metro.en_hizli_rota_bul("M1", "K4")
if sonuc:
    rota, sure = sonuc
    print(f"En hızlı rota ({sure} dakika):", " -> ".join(i.ad for i in rota))


1. AŞTİ'den OSB'ye:
En az aktarmalı rota: AŞTİ -> Kızılay -> Kızılay -> Ulus -> Demetevler -> OSB
En hızlı rota (25 dakika): AŞTİ -> Kızılay -> Kızılay -> Ulus -> Demetevler -> OSB


#### *Senaryo 2: Batıkent'ten Keçiören'e*

In [ ]:
print("\n2. Batıkent'ten Keçiören'e:")
rota = metro.en_az_aktarma_bul("T1", "T4")
if rota:
    print("En az aktarmalı rota:", " -> ".join(i.ad for i in rota))

sonuc = metro.en_hizli_rota_bul("T1", "T4")
if sonuc:
    rota, sure = sonuc
    print(f"En hızlı rota ({sure} dakika):", " -> ".join(i.ad for i in rota))


2. Batıkent'ten Keçiören'e:
En az aktarmalı rota: Batıkent -> Demetevler -> Gar -> Keçiören
En hızlı rota (21 dakika): Batıkent -> Demetevler -> Gar -> Keçiören


#### *Senaryo 3: Keçiören'den AŞTİ'ye*

In [ ]:
print("\n3. Keçiören'den AŞTİ'ye:")
rota = metro.en_az_aktarma_bul("T4", "M1")
if rota:
    print("En az aktarmalı rota:", " -> ".join(i.ad for i in rota))

sonuc = metro.en_hizli_rota_bul("T4", "M1")
if sonuc:
    rota, sure = sonuc
    print(f"En hızlı rota ({sure} dakika):", " -> ".join(i.ad for i in rota))


3. Keçiören'den AŞTİ'ye:
En az aktarmalı rota: Keçiören -> Gar -> Gar -> Sıhhiye -> Kızılay -> AŞTİ
En hızlı rota (19 dakika): Keçiören -> Gar -> Gar -> Sıhhiye -> Kızılay -> AŞTİ


#### *Senaryo 4: Daha az aktarmalı ama daha uzun sürebilecek bir rota*

In [33]:
print("\n4. AŞTİ’den Keçiören’e:")
rota = metro.en_az_aktarma_bul("K2", "T4")  # Daha az aktarmalı ama daha uzun sürebilecek bir rota
if rota:
    print("En az aktarmalı rota:", " -> ".join(i.ad for i in rota))

sonuc = metro.en_hizli_rota_bul("K2", "T4")  # Daha fazla aktarma ile ama daha kısa sürede ulaşılabilecek bir rota
if sonuc:
    rota, sure = sonuc
    print(f"En hızlı rota ({sure} dakika):", " -> ".join(i.ad for i in rota))


4. AŞTİ’den Keçiören’e:
En az aktarmalı rota: Ulus -> Demetevler -> Demetevler -> Gar -> Keçiören
En hızlı rota (20 dakika): Ulus -> Kızılay -> Kızılay -> Sıhhiye -> Gar -> Gar -> Keçiören


#### *Senaryo 5: Hedef İstasyonun Bulunmadığı Durum (Geçersiz İstasyon)*

In [ ]:
print("\n5. Hedef İstasyon Bulunamadı (Geçersiz İstasyon):")
rota = metro.en_az_aktarma_bul("M1", "X3")  # X3 geçersiz bir istasyon
if rota:
    print("En az aktarmalı rota:", " -> ".join(i.ad for i in rota))

sonuc = metro.en_hizli_rota_bul("M1", "X3")  # X3 geçersiz bir istasyon
if sonuc:
    rota, sure = sonuc
    print(f"En hızlı rota ({sure} dakika):", " -> ".join(i.ad for i in rota))


5. Hedef İstasyon Bulunamadı (Geçersiz İstasyon):
Hedef istasyonu ağda bulunamadı.
Hedef istasyonu ağda bulunamadı.


#### *Senaryo 6: Geçersiz Başlangıç İstasyonu*

In [34]:
print("\n6. Geçersiz Başlangıç İstasyonu (Başlangıç yok):")
rota = metro.en_az_aktarma_bul("X1", "K4")  # X1 geçersiz bir başlangıç istasyonu
if rota:
    print("En az aktarmalı rota:", " -> ".join(i.ad for i in rota))

sonuc = metro.en_hizli_rota_bul("X1", "K4")  # X1 geçersiz bir başlangıç istasyonu
if sonuc:
    rota, sure = sonuc
    print(f"En hızlı rota ({sure} dakika):", " -> ".join(i.ad for i in rota))


6. Geçersiz Başlangıç İstasyonu (Başlangıç yok):
Başlangıç istasyonu ağda bulunamadı.
Başlangıç istasyonu ağda bulunamadı.


#### *Senaryo 7: Aynı İstasyon, Başlangıç ve Hedef Aynı*

In [35]:
print("\n7. Başlangıç ve Hedef Aynı (AŞTİ'den AŞTİ'ye):")
rota = metro.en_az_aktarma_bul("M1", "M1")  # Aynı istasyona gitmek
if rota:
    print("En az aktarmalı rota:", " -> ".join(i.ad for i in rota))

sonuc = metro.en_hizli_rota_bul("M1", "M1")  # Aynı istasyona gitmek
if sonuc:
    rota, sure = sonuc
    print(f"En hızlı rota ({sure} dakika):", " -> ".join(i.ad for i in rota))


7. Başlangıç ve Hedef Aynı (AŞTİ'den AŞTİ'ye):
En az aktarmalı rota: AŞTİ
En hızlı rota (0 dakika): AŞTİ


#### *Senaryo 8: Ters Rota (İstasyonlar Arasında Ters Yön)*

In [36]:
print("\n8. Ters Yönle Rota Arama (AŞTİ'den Keçiören'e):")
rota = metro.en_az_aktarma_bul("M1", "T4")  # AŞTİ'den Keçiören'e, ters yön
if rota:
    print("En az aktarmalı rota:", " -> ".join(i.ad for i in rota))

sonuc = metro.en_hizli_rota_bul("M1", "T4")  # AŞTİ'den Keçiören'e, ters yön
if sonuc:
    rota, sure = sonuc
    print(f"En hızlı rota ({sure} dakika):", " -> ".join(i.ad for i in rota))


8. Ters Yönle Rota Arama (AŞTİ'den Keçiören'e):
En az aktarmalı rota: AŞTİ -> Kızılay -> Sıhhiye -> Gar -> Gar -> Keçiören
En hızlı rota (19 dakika): AŞTİ -> Kızılay -> Sıhhiye -> Gar -> Gar -> Keçiören


## 4-Bonus

### 4.1-BFS Detaylı Anlatım

Başlangıç: Ulus (K2)

Hedef: Keçiören (T2)

🔹 BFS Algoritması ile en az aktarmalı rotayı bulacağız.

📌 İstasyonlar ve Bağlantılar:

- Ulus (K2) ↔ Demetevler (K3) (6 dk)
- Demetevler (K3) ↔ Demetevler (T2) (3 dk)

Toplam süre: 2 aktarma  9 dakika 

<div>
<img src="../diagrams/istasyonlar.svg" width="750"/>
</div>

### 4.1.1-Kuyruğumuzun başlangıç durumu


kuyruk = deque([(K2, ["Ulus"])])

deque([

    (K2, ["Ulus"])

])

Ziyaret Edilenler: {K2}

### 4.1.2-K2 İşlenir

mevcut, rota = kuyruk.popleft()

🟢 Mevcut istasyon: M1 (Ulus)

🔹 Şu ana kadar izlenen rota: ["Ulus"]

K2'nin komşuları:

- K1 (Kızılay, 4 dk)
- K3 (Demetevler, 6 dk)

📌 Yeni Kuyruk:

deque([
    
    (K1, [K2 → K1]), 
    (K3, [K2 → K3])
    
])

Ziyaret Edilenler: {K2, K1, K3}

### 4.1.3-K1 İşlenir

mevcut, rota = kuyruk.popleft()

K1'in komşuları:

- M2 (Kızılay, aktarma)

📌 Yeni Kuyruk:

deque([

    (K3, [K2 → K3]), 
    (M2, [K2 → K1 → M2])

])

Ziyaret Edilenler: {K2, K1, K3, M2}

### 4.1.4-K3 İşlenir

mevcut, rota = kuyruk.popleft()

K3'ün komşuları:

- K4 (OSB)
- T2 (Demetevler)

deque([
    
    (M2, [K2 → K1 → M2]), 
    (K4, [K2 → K3 → K4]), 
    (T2, [K2 → K3 → T2])
    
])

Ziyaret Edilenler: {K2, K1, K3, M2, K4, T2}

T2 bulundu en hızlı rota 

*K2 → K3 → T2* 

## 4.2 A* Detaylı Anlatım

1️⃣ Algoritma Başlangıcı

- Başlangıç: K2 (Ulus, Kırmızı Hat)
- Hedef: T2 (Keçiören, Turuncu Hat)
- Öncelik Sırası (Min-Heap) başlatılır:

pq = [(
    
    0, Toplam maliyet
    0, heuristik maliyet
    K2, mevcut istasyon
    [K2] rota
)]  

Ziyaret Edilenler: {}

### 4.2.1 K2 İşlenir

Komşular:

- K1 (Kızılay, 4 dk)
- K3 (Demetevler, 6 dk)

pq = [(4 + h(K1), K1, [K2 → K1]), (6 + h(K3), K3, [K2 → K3])]

Ziyaret Edilenler: {K2}

### 4.2.2 K1 İşlenir

Komşular:

- M2 (Kızılay, Mavi Hat, 2 dk aktarma)

pq = [(6 + h(K3), K3, [K2 → K3]), (6 + h(M2), M2, [K2 → K1 → M2])]

Ziyaret Edilenler: {K2, K1}

### 4.2.3 K3 İşlenir

Komşular:

- K4 (OSB, 8 dk)
- T2 (Demetevler, 3 dk aktarma) → Hedefe ulaştık! ✅

pq = [(6 + h(M2), M2, [K2 → K1 → M2]), (9 + h(T2), T2, [K2 → K3 → T2])]

Sonuç: K2 → K3 → T2